# Data Processing model

In the we read the dataset and process the basics. In dataset, we wil add a faiss index to the dataset. 

In [11]:
# Read the UMT english weights 
import glob
import gc
from datasets import load_dataset
streamingqa_location = sorted(glob.glob("/home/saisur/streamingqa/*"))
streamingqa_location

['/home/saisur/streamingqa/streaminqa_eval.jsonl',
 '/home/saisur/streamingqa/streaminqa_train.jsonl',
 '/home/saisur/streamingqa/streaminqa_valid.jsonl']

In [15]:
streamqa_dataset = load_dataset("json", data_files= streamingqa_location[1])
test = load_dataset("json", data_files= streamingqa_location[0])
valid = load_dataset("json", data_files= streamingqa_location[2])    
streamqa_dataset["test"] = test["train"]
streamqa_dataset["validation"] = valid["train"]
del test, valid
gc.collect()
streamqa_dataset


Using custom data configuration default-147dff59dd9a83af
Found cached dataset json (/home/saisur/.cache/huggingface/datasets/json/default-147dff59dd9a83af/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-61e6bb4d9647ad78
Found cached dataset json (/home/saisur/.cache/huggingface/datasets/json/default-61e6bb4d9647ad78/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-a099a47761949eb4
Found cached dataset json (/home/saisur/.cache/huggingface/datasets/json/default-a099a47761949eb4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 99402
    })
    test: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 36378
    })
    validation: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_seve

In [16]:
from datasets import load_from_disk
dataset = load_from_disk("/home/saisur/nlg-retroweights/filtered")
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'sentence_split', 'non_split', 'hash'],
        num_rows: 19863186
    })
})

In [21]:
dataset["train"][0].keys()

dict_keys(['date', 'sentence_split', 'non_split', 'hash'])

In [25]:
import base64

def convert_base64(example):
    example["sentence_split"] = base64.b64decode(example["sentence_split"])  
    example["non_split"] = base64.b64decode(example["non_split"])  
    return example


In [26]:
dataset = dataset.map(convert_base64, batched=False, num_proc=10)

#0:   0%|          | 0/1986319 [00:00<?, ?ex/s]

#5:   0%|          | 0/1986319 [00:00<?, ?ex/s]

#2:   0%|          | 0/1986319 [00:00<?, ?ex/s]

#3:   0%|          | 0/1986319 [00:00<?, ?ex/s]

#1:   0%|          | 0/1986319 [00:00<?, ?ex/s]

#4:   0%|          | 0/1986319 [00:00<?, ?ex/s]

#7:   0%|          | 0/1986318 [00:00<?, ?ex/s]

#9:   0%|          | 0/1986318 [00:00<?, ?ex/s]

#6:   0%|          | 0/1986318 [00:00<?, ?ex/s]

#8:   0%|          | 0/1986318 [00:00<?, ?ex/s]

In [31]:
dataset["train"][0]

{'date': 20080101,
 'sentence_split': "Oregon Routs South Florida to End Losing Streak\nFiled at 6:36 p.m.\nEL PASO, Texas (AP) -- Jonathan Stewart set a Sun Bowl record with a career-high 253 yards rushing and Justin Roper threw four touchdown passes in his first start as Oregon beat South Florida 56-21 on Monday.\nThe Ducks (9-4) scored 28 straight points in the third quarter to snap a four-game postseason losing streak, their first bowl win since beating Colorado 38-16 in the Fiesta Bowl after the 2001 season.\nOregon made it look easy against the Bulls (9-4), who boasted one of the nation's better defenses.\nCredit the steady play by Roper, a redshirt freshman, and Stewart, as smooth a runner as you'll find.\nThe Ducks won for the first time since losing quarterback Dennis Dixon to a knee injury in November, snapping a three-game losing streak the killed their national title hopes.\nHow easy was it for the Ducks?\nThey won a matchup of teams that'd been ranked No.\n2 in the nation 

In [38]:
streamqa_dataset["train"][0]["evidence_id"].split("\x00\x01")

['20170305000000000000',
 '1c98344aaf81280d4f1dae80f8ca31a6318a40040d7f2ce803ff455959800723',
 '']

In [48]:
train_evidemceid = streamqa_dataset["train"]["evidence_id"]
test_evidemceid = streamqa_dataset["test"]["evidence_id"]
valid_evidemceid = streamqa_dataset["validation"]["evidence_id"]
all_evidence = train_evidemceid + test_evidemceid + valid_evidemceid
all_evidence = [x.split("\x00\x01")[1] for x in all_evidence]

In [50]:
dataset_f = dataset.filter(lambda x: x["hash"] in all_evidence, num_proc=10)

#1:   0%|          | 0/1987 [00:00<?, ?ba/s]

#6:   0%|          | 0/1987 [00:00<?, ?ba/s]

#7:   0%|          | 0/1987 [00:00<?, ?ba/s]

#2:   0%|          | 0/1987 [00:00<?, ?ba/s]

#0:   0%|          | 0/1987 [00:00<?, ?ba/s]

#5:   0%|          | 0/1987 [00:00<?, ?ba/s]

#3:   0%|          | 0/1987 [00:00<?, ?ba/s]

#4:   0%|          | 0/1987 [00:00<?, ?ba/s]

#8:   0%|          | 0/1987 [00:00<?, ?ba/s]

#9:   0%|          | 0/1987 [00:00<?, ?ba/s]

In [ ]:
dataset_f

['...',
 ...]